In [1]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
# text in Western (Windows 1252)

import pickle
import numpy as np
# import StringIO
import math
from keras import optimizers, metrics
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import concatenate
from keras import regularizers
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.constraints import maxnorm
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.models import load_model
# from keras import backend as Input
np.random.seed(7)

Using Theano backend.


In [2]:
# %run ../../../prepare_data.py

import sys
sys.path.insert(0, '../../../')
from prepare_data import *

In [3]:
# %run ../../../prepare_data.py
# X_train, X_other_features_train, y_train, X_test, X_other_features_test, y_test, X_validate, X_other_features_validate, y_validate = generate_syllable_inputs('../../internal_representations/inputs/content_shuffle_vector.h5', '../../internal_representations/inputs/shuffle_vector')
# save_inputs('../../internal_representations/inputs/shuffeled_syllable_train_inputs.h5', X_train, y_train, other_features = X_other_features_train)
# save_inputs('../../internal_representations/inputs/shuffeled_syllable_test_inputs.h5', X_test, y_test, other_features = X_other_features_test)
# save_inputs('../../internal_representations/inputs/shuffeled_syllable_validate_inputs.h5', X_validate, y_validate,  other_features = X_other_features_validate)
# X_train, X_other_features_train, y_train = load_inputs('../../internal_representations/inputs/shuffeled_syllable_train_inputs.h5', other_features=True)
# X_test, X_other_features_test, y_test = load_inputs('../../internal_representations/inputs/shuffeled_syllable_test_inputs.h5', other_features=True)
# X_validate, X_other_features_validate, y_validate = load_inputs('../../internal_representations/inputs/shuffeled_syllable_validate_inputs.h5', other_features=True)

# syllable_letters_translator = create_syllable_letters_translator(max_syllable, syllable_dictionary, dictionary, vowels)

data = Data('s', accent_classification=True)
data.generate_data('syllabled_letters_accent_classification_train',
                   'syllabled_letters_accent_classification_test',
                   'syllabled_letters_accent_classification_validate', force_override=False)

LOADING DATA...
LOAD SUCCESSFUL!


In [4]:
# concatenate test and train data
data.x_train = np.concatenate((data.x_train, data.x_test), axis=0)
data.x_other_features_train = np.concatenate((data.x_other_features_train, data.x_other_features_test), axis=0)
data.y_train = np.concatenate((data.y_train, data.y_test), axis=0)

In [8]:
%run ../../../prepare_data.py
gen = data.generator('train', 16)
test = next(gen)

In [21]:
# prints test word from generator
# print(X_train[0])
# print(syllable_dictionary[test[0][0][0][0].tolist().index(1)])
print(np.array(test[0][0]).shape)
for el in test[0][0][0]:
    syllable = ''
    for pos in range(0, len(el), 36):
        syllable += data.decode_x([el[pos:pos+36]], dictionary)
    print(syllable)
#     print(el)
    
#     print(syllable_dictionary[el.tolist().index(1)])

(16, 11, 252)
na
ja
ktu
a
lnej
ša







In [5]:
num_examples = len(data.x_train)

nn_output_dim = 13
nn_hdim = 516
batch_size = 16
actual_epoch = 20
num_fake_epoch = 20



#conv_input_shape=(10, 252)
conv_input_shape=(10, 5168)
othr_input = (150, )

conv_input = Input(shape=conv_input_shape, name='conv_input')
# x_conv = Conv1D(43, (3), padding='same', activation='relu')(conv_input)
x_conv = Conv1D(200, (2), padding='same', activation='relu')(conv_input)
x_conv = MaxPooling1D(pool_size=2)(x_conv)
x_conv = Flatten()(x_conv)
# x_conv = Dense(516, activation='relu', kernel_constraint=maxnorm(3))(x_conv)

othr_input = Input(shape=othr_input, name='othr_input')
# x_othr = Dense(256, input_dim=167, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(512, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(256, activation='relu')(othr_input)

x = concatenate([x_conv, othr_input])
# x = Dense(1024, input_dim=(516 + 256), activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)




model = Model(inputs=[conv_input, othr_input], outputs=x)
opt = optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[actual_accuracy,])

In [6]:
y_array_train = np.asarray(data.y_train)
accentuation_length_train = (y_array_train > 0).sum()
y_array_test = np.asarray(data.y_test)
accentuation_length_test = (y_array_test > 0).sum()
history = model.fit_generator(data.generator('train', batch_size), 
                              accentuation_length_train/(batch_size * num_fake_epoch), 
                              epochs=actual_epoch*num_fake_epoch)

Epoch 1/400
1625/1624 [==============================] - 293s - loss: 0.2248 - actual_accuracy: 0.0756   
Epoch 2/400
1625/1624 [==============================] - 342s - loss: 0.1321 - actual_accuracy: 0.4898   
Epoch 3/400
1625/1624 [==============================] - 342s - loss: 0.0922 - actual_accuracy: 0.6678   
Epoch 4/400
1625/1624 [==============================] - 350s - loss: 0.0710 - actual_accuracy: 0.7631   
Epoch 5/400
1625/1624 [==============================] - 362s - loss: 0.0556 - actual_accuracy: 0.8166   
Epoch 6/400
1625/1624 [==============================] - 376s - loss: 0.0460 - actual_accuracy: 0.8483   
Epoch 7/400
1625/1624 [==============================] - 386s - loss: 0.0386 - actual_accuracy: 0.8720   
Epoch 8/400
1625/1624 [==============================] - 387s - loss: 0.0335 - actual_accuracy: 0.8915   
Epoch 9/400
1625/1624 [==============================] - 394s - loss: 0.0292 - actual_accuracy: 0.9059   
Epoch 10/400
1625/1624 [======================

1625/1624 [==============================] - 554s - loss: 0.0022 - actual_accuracy: 0.9938   
Epoch 78/400
1625/1624 [==============================] - 555s - loss: 0.0025 - actual_accuracy: 0.9929   
Epoch 79/400
1625/1624 [==============================] - 553s - loss: 0.0025 - actual_accuracy: 0.9926   
Epoch 80/400
1625/1624 [==============================] - 557s - loss: 0.0016 - actual_accuracy: 0.9954   
Epoch 81/400
1625/1624 [==============================] - 552s - loss: 0.0018 - actual_accuracy: 0.9947   
Epoch 82/400
1625/1624 [==============================] - 552s - loss: 0.0019 - actual_accuracy: 0.9945   
Epoch 83/400
1625/1624 [==============================] - 556s - loss: 0.0019 - actual_accuracy: 0.9945   
Epoch 84/400
1625/1624 [==============================] - 555s - loss: 0.0020 - actual_accuracy: 0.9945   
Epoch 85/400
1625/1624 [==============================] - 554s - loss: 0.0019 - actual_accuracy: 0.9949   
Epoch 86/400
1625/1624 [==========================

1625/1624 [==============================] - 610s - loss: 7.6835e-04 - actual_accuracy: 0.9978   
Epoch 153/400
1625/1624 [==============================] - 607s - loss: 0.0010 - actual_accuracy: 0.9970   
Epoch 154/400
1625/1624 [==============================] - 603s - loss: 9.6153e-04 - actual_accuracy: 0.9972   
Epoch 155/400
1625/1624 [==============================] - 593s - loss: 8.2502e-04 - actual_accuracy: 0.9980   
Epoch 156/400
1625/1624 [==============================] - 598s - loss: 0.0010 - actual_accuracy: 0.9979   
Epoch 157/400
1625/1624 [==============================] - 595s - loss: 9.5121e-04 - actual_accuracy: 0.9972   
Epoch 158/400
1625/1624 [==============================] - 603s - loss: 0.0013 - actual_accuracy: 0.9970   
Epoch 159/400
1625/1624 [==============================] - 620s - loss: 0.0010 - actual_accuracy: 0.9970   
Epoch 160/400
1625/1624 [==============================] - 634s - loss: 8.1211e-04 - actual_accuracy: 0.9981   
Epoch 161/400
1625/162

1625/1624 [==============================] - 609s - loss: 7.0359e-04 - actual_accuracy: 0.9981   
Epoch 226/400
1625/1624 [==============================] - 606s - loss: 7.4193e-04 - actual_accuracy: 0.9983   
Epoch 227/400
1625/1624 [==============================] - 612s - loss: 8.2989e-04 - actual_accuracy: 0.9977   
Epoch 228/400
1625/1624 [==============================] - 615s - loss: 6.7409e-04 - actual_accuracy: 0.9981   
Epoch 229/400
1625/1624 [==============================] - 613s - loss: 5.5222e-04 - actual_accuracy: 0.9983   
Epoch 230/400
1625/1624 [==============================] - 778s - loss: 7.2789e-04 - actual_accuracy: 0.9980   
Epoch 231/400
1625/1624 [==============================] - 616s - loss: 5.2232e-04 - actual_accuracy: 0.9983   
Epoch 232/400
1625/1624 [==============================] - 624s - loss: 5.3549e-04 - actual_accuracy: 0.9984   
Epoch 233/400
1625/1624 [==============================] - 627s - loss: 8.2793e-04 - actual_accuracy: 0.9979   
Epoch 

1625/1624 [==============================] - 672s - loss: 6.7364e-04 - actual_accuracy: 0.9979   
Epoch 299/400
1625/1624 [==============================] - 650s - loss: 6.8566e-04 - actual_accuracy: 0.9984   
Epoch 300/400
1625/1624 [==============================] - 641s - loss: 6.5032e-04 - actual_accuracy: 0.9982   
Epoch 301/400
1625/1624 [==============================] - 649s - loss: 6.0326e-04 - actual_accuracy: 0.9985   
Epoch 302/400
1625/1624 [==============================] - 634s - loss: 8.1198e-04 - actual_accuracy: 0.9981   
Epoch 303/400
1625/1624 [==============================] - 650s - loss: 6.3953e-04 - actual_accuracy: 0.9985   
Epoch 304/400
1625/1624 [==============================] - 635s - loss: 5.8099e-04 - actual_accuracy: 0.9985   
Epoch 305/400
1625/1624 [==============================] - 643s - loss: 6.6787e-04 - actual_accuracy: 0.9980   
Epoch 306/400
1625/1624 [==============================] - 640s - loss: 6.0106e-04 - actual_accuracy: 0.9984   
Epoch 

1625/1624 [==============================] - 655s - loss: 7.0761e-04 - actual_accuracy: 0.9985   
Epoch 372/400
1625/1624 [==============================] - 652s - loss: 5.7858e-04 - actual_accuracy: 0.9984   
Epoch 373/400
1625/1624 [==============================] - 650s - loss: 6.1080e-04 - actual_accuracy: 0.9982   
Epoch 374/400
1625/1624 [==============================] - 654s - loss: 5.9091e-04 - actual_accuracy: 0.9985   
Epoch 375/400
1625/1624 [==============================] - 649s - loss: 4.9408e-04 - actual_accuracy: 0.9987   
Epoch 376/400
1625/1624 [==============================] - 655s - loss: 6.3690e-04 - actual_accuracy: 0.9981   
Epoch 377/400
1625/1624 [==============================] - 651s - loss: 5.2863e-04 - actual_accuracy: 0.9987   
Epoch 378/400
1625/1624 [==============================] - 656s - loss: 6.4666e-04 - actual_accuracy: 0.9987   
Epoch 379/400
1625/1624 [==============================] - 649s - loss: 6.7678e-04 - actual_accuracy: 0.9984   
Epoch 

In [7]:
name = '20_test_epoch'
model.save(name + '.h5')
output = open(name + '_history.pkl', 'wb')
pickle.dump(history.history, output)
output.close()

In [15]:
# dictionary, max_word, max_num_vowels, content, vowels, accetuated_vowels = create_dict()
# train_content, test_content, validate_content = split_content(content, 0.2, '../../internal_representations/inputs/content_shuffle_vector.h5')
# feature_dictionary = create_feature_dictionary()
# syllable_dictionary = create_syllables_dictionary(content, vowels)
# max_syllable = get_max_syllable(syllable_dictionary)


content = data._read_content('../../../data/SlovarIJS_BESEDE_utf8.lex')
dictionary, max_word, max_num_vowels, vowels, accented_vowels = data._create_dict(content)
feature_dictionary = data._create_feature_dictionary()
syllable_dictionary = data._create_syllables_dictionary(content, vowels)

In [8]:
%run ../../../prepare_data.py
# generate_X_and_y(dictionary, max_word, max_num_vowels, content, vowels, accetuated_vowels, feature_dictionary)